In [1]:
%load_ext autoreload
%autoreload 2
%matplotlib widget
import os, sys, getpass
user = getpass.getuser()
sys.path.append(f'/home/{user}/codes/hpe_library/')
from lib_import import *
from my_utils import *
os.chdir('/home/hrai/codes/MotionBERT')

from lib.data.datareader_total import DataReaderTotal
from lib.data.datareader_h36m import DataReaderH36M
from lib.data.datareader_fit3d import DataReaderFIT3D
from lib.utils.args import get_opts_args, get_opt_args_from_model_name
from lib.model.load_model import load_model
from lib.model.load_dataset import load_dataset
from lib.data.dataset_motion_3d import MotionDataset3DTotal
from lib.model.evaluation import *

In [2]:
checkpoint = 'MB_train_h36m_gt_cam_no_factor_s15678_tr_54138969_ts_others'
args, opts = get_opt_args_from_model_name(checkpoint)
model_pos, chk_filename, checkpoint = load_model(opts, args)

configs/pose3d/MB_train_h36m_gt_cam_no_factor_s15678_tr_54138969_ts_others.yaml
True
MB_train_h36m_gt_cam_no_factor_s15678_tr_54138969_ts_others
Loading checkpoint checkpoint/pose3d/MB_train_h36m_gt_cam_no_factor_s15678_tr_54138969_ts_others/best_epoch.bin


In [7]:
subset = 'H36M-GT-CAM_NO_FACTOR-S15678_TR_54138969_TS_OTHERS'
args.subset_list = [subset]
train_loader, test_loader, _, _, datareader = load_dataset(args, use_new_datareader=True)

Loading dataset...
H36M-GT-CAM_NO_FACTOR-S15678_TR_54138969_TS_OTHERS
==> Loading H36M source_list... overwrite: False
==> Loading H36M cam_param... overwrite: False
==> Loading H36M cam_3d... overwrite: False
==> Loading H36M img_2d... overwrite: False


In [ ]:
if '3DHP' in subset:
    args.eval_part = 'whole_without_nose'
    args.part_list = ['whole_without_nose']

In [6]:
e1, e2, results_all, inputs_all, gts_all, total_result_dict = evaluate(args, model_pos, test_loader, datareader, checkpoint, only_one_batch=False)

INFO: Testing
No epoch information in the checkpoint


  0%|          | 0/287 [00:00<?, ?it/s]

100%|██████████| 287/287 [01:35<00:00,  2.99it/s]


No eval_keypoint. Use part list
Part: whole
+------------+-------------------+--------------------+-------------------+-------------------+--------------------+-------------------+--------------------+--------------------+-------------------+------------------+--------------------+-------------------+-------------------+-------------------+-------------------+-------------------+-------------------+--------------------+-------------------+-------------------+-------------------+-------------------+-------------------+-------------------+--------------------+------------------+--------------------+--------------------+--------------------+-------------------+-------------------+-------------------+-------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+-------------------+-------------------+--------------------+--------------------+--------------------+
| test_name  |     Directions    |    Direct

### save total results

In [ ]:
config_root = 'configs/pose3d/'
checkpoint_root = 'checkpoint/pose3d/'

blacklist = ['MB_ft_h36m']
if os.path.exists('result_dict.pkl'):
    result_dict = readpkl('result_dict.pkl')
else:
    result_dict = {}
for i, item in enumerate(natsorted(glob('/home/hrai/codes/MotionBERT/custom_codes/evaluation/experiments/*.txt'))):
    checkpoint = os.path.basename(item).split('.')[0]
    if checkpoint in blacklist: continue
    if 'cam_no_factor' not in checkpoint: continue
    if not os.path.exists(checkpoint_root+checkpoint+'/best_epoch.bin'): 
        print(f'{checkpoint} not exists')
        continue
    config = checkpoint + '.yaml'
    input_args = ['--config', config_root+config, '--evaluate', checkpoint_root+checkpoint+'/best_epoch.bin']
    args, opts = get_opts_args(input_args, verbose=False)
    args.print_summary_table = False
    
    testloader_params = {
        'batch_size': args.batch_size,
        'shuffle': False,
        'num_workers': 12,
        'pin_memory': True,
        'prefetch_factor': 4,
        'persistent_workers': True
    }
    
    model_pos, _, _ = load_model(opts, args, verbose=False)
    print(f'[{checkpoint}] ({i+1}/{len(glob("/home/hrai/codes/MotionBERT/custom_codes/evaluation/experiments/*.txt"))})')
    if checkpoint not in result_dict.keys(): result_dict[checkpoint] = {}
    with open(item, 'r') as file:
        lines = file.readlines()
    subset_list = [line.split('\n')[0] for line in lines]
    for subset in subset_list:
        print(subset)
        if subset not in result_dict[checkpoint].keys(): result_dict[checkpoint][subset] = {}
        else:
            if 'e1' in result_dict[checkpoint][subset].keys() and 'e2' in result_dict[checkpoint][subset].keys(): 
                if result_dict[checkpoint][subset]['e1'] != None and result_dict[checkpoint][subset]['e2'] != None: continue
        try:
            datareader = DataReaderTotal(n_frames=243, 
                                        sample_stride=1, 
                                        data_stride_train=81, 
                                        data_stride_test=243, 
                                        subset=subset,
                                        verbose=False)
            train_data, test_data, train_labels, test_labels = datareader.get_sliced_data()
            test_dataset = MotionDataset3DTotal(args, test_data.copy(), test_labels.copy(), 'test')
            test_loader = DataLoader(test_dataset, **testloader_params)
            e1, e2, results_all, inputs_all, gts_all, total_result_dict = evaluate(args, model_pos, test_loader, datareader, checkpoint, only_one_batch=False, verbose=False)
            result_dict[checkpoint][subset] = {'e1': e1, 'e2': e2}
        except Exception as e:
            print(e)
            result_dict[checkpoint][subset] = {'e1': None, 'e2': None, 'error': str(e)}
        #break
    #break
#savepkl(result_dict, 'result_dict.pkl')

In [ ]:
savepkl(result_dict, 'result_dict.pkl')